In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
 import pickle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD
from keras.layers.convolutional import Convolution2D, MaxPooling2D
import numpy as np
import keras
from keras.utils.np_utils import to_categorical
import time
from keras.callbacks import EarlyStopping

In [0]:
f = open("./drive/ColabNotebooks/Eye/train_data.pkl", "rb")
obj = pickle.load(f)

In [31]:
obj[0].shape, obj[1].shape

((4846, 24, 24), (4846,))

In [0]:
X_train, X_test, y_train, y_test = train_test_split(obj[0], obj[1], test_size = 0.1, random_state = 42)

In [0]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size= 0.05, random_state = 42)

In [34]:
print("Data: ")
print("Train Data: ", len(X_train))
print("Valid Data: ", len(X_valid))
print("Test Data: ", len(X_test))

Data: 
Train Data:  4142
Valid Data:  219
Test Data:  485


In [35]:
# X_train = X_train.reshape((X_train.shape[0], 1) + X_train.shape[1:3])
# X_test = np.reshape((X_test.shape[0], 1) + X_test.shape[1:3])
# X_valid = np.reshape((X_valid.shape[0], 1) + X_valid.shape[1:3])
X_train = np.reshape(X_train, newshape=(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
X_valid = np.reshape(X_valid, newshape=(X_valid.shape[0], X_valid.shape[1], X_valid.shape[2], 1))
X_test = np.reshape(X_test, newshape=(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1))

y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)
y_valid = to_categorical(y_valid, num_classes=2)

print(X_train.shape, y_train.shape)

(4142, 24, 24, 1) (4142, 2)


In [0]:
_, _img_rows, _img_cols, _channels = X_train.shape

In [60]:
model = Sequential()

#Input
model.add(Convolution2D(32, (3, 3), padding='same',
                        input_shape=(_img_rows, _img_cols, _channels)))
model.add(Activation('relu'))
model.add(Convolution2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Convolution2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(2))
model.add(Activation('sigmoid'))

sgd = SGD(lr = 1e-3, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

# define early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=20, mode='auto')
callbacks_list = [earlystop]

batch_size = 128
epochs = 200

model.fit(X_train, y_train, batch_size = batch_size, epochs=epochs, validation_data=(X_valid, y_valid), callbacks=callbacks_list)

score = model.evaluate(X_test, y_test)
print("Test score: ", score)


Train on 4142 samples, validate on 219 samples
Epoch 1/200
4142/4142 [==============================] - 4s 890us/step - loss: 0.9140 - acc: 0.4971 - val_loss: 0.6903 - val_acc: 0.4726
Epoch 2/200
4142/4142 [==============================] - 1s 291us/step - loss: 0.6924 - acc: 0.4810 - val_loss: 0.6899 - val_acc: 0.5046
Epoch 3/200
4142/4142 [==============================] - 1s 282us/step - loss: 0.6918 - acc: 0.4934 - val_loss: 0.6899 - val_acc: 0.5388
Epoch 4/200
4142/4142 [==============================] - 1s 285us/step - loss: 0.6910 - acc: 0.4989 - val_loss: 0.6880 - val_acc: 0.5251
Epoch 5/200
4142/4142 [==============================] - 1s 287us/step - loss: 0.6895 - acc: 0.5027 - val_loss: 0.6858 - val_acc: 0.5457
Epoch 6/200
4142/4142 [==============================] - 1s 283us/step - loss: 0.6885 - acc: 0.5046 - val_loss: 0.6826 - val_acc: 0.5274
Epoch 7/200
4142/4142 [==============================] - 1s 281us/step - loss: 0.6859 - acc: 0.5022 - val_loss: 0.6764 - val_acc: 0

4142/4142 [==============================] - 1s 283us/step - loss: 0.6781 - acc: 0.5130 - val_loss: 0.6581 - val_acc: 0.5753
Epoch 10/200
4142/4142 [==============================] - 1s 281us/step - loss: 0.6701 - acc: 0.5227 - val_loss: 0.6540 - val_acc: 0.6826
Epoch 11/200
4142/4142 [==============================] - 1s 285us/step - loss: 0.6633 - acc: 0.5410 - val_loss: 0.6300 - val_acc: 0.7420
Epoch 12/200
4142/4142 [==============================] - 1s 285us/step - loss: 0.6450 - acc: 0.5930 - val_loss: 0.5930 - val_acc: 0.8151
Epoch 13/200
4142/4142 [==============================] - 1s 284us/step - loss: 0.6103 - acc: 0.6696 - val_loss: 0.5304 - val_acc: 0.8402
Epoch 14/200
4142/4142 [==============================] - 1s 278us/step - loss: 0.5462 - acc: 0.7419 - val_loss: 0.4305 - val_acc: 0.8311
Epoch 15/200
4142/4142 [==============================] - 1s 276us/step - loss: 0.4864 - acc: 0.7814 - val_loss: 0.4395 - val_acc: 0.8037
Epoch 16/200
4142/4142 [=======================

4142/4142 [==============================] - 1s 289us/step - loss: 0.4044 - acc: 0.8315 - val_loss: 0.3187 - val_acc: 0.8767
Epoch 19/200
4142/4142 [==============================] - 1s 285us/step - loss: 0.3761 - acc: 0.8460 - val_loss: 0.2805 - val_acc: 0.8973
Epoch 20/200
4142/4142 [==============================] - 1s 279us/step - loss: 0.3505 - acc: 0.8582 - val_loss: 0.2548 - val_acc: 0.8973
Epoch 21/200
4142/4142 [==============================] - 1s 290us/step - loss: 0.3310 - acc: 0.8662 - val_loss: 0.2275 - val_acc: 0.9384
Epoch 22/200
4142/4142 [==============================] - 1s 282us/step - loss: 0.3147 - acc: 0.8739 - val_loss: 0.2419 - val_acc: 0.9018
Epoch 23/200
4142/4142 [==============================] - 1s 279us/step - loss: 0.2928 - acc: 0.8873 - val_loss: 0.1930 - val_acc: 0.9361
Epoch 24/200
4142/4142 [==============================] - 1s 288us/step - loss: 0.2939 - acc: 0.8844 - val_loss: 0.1950 - val_acc: 0.9406
Epoch 25/200
4142/4142 [=======================

4142/4142 [==============================] - 1s 284us/step - loss: 0.2511 - acc: 0.9029 - val_loss: 0.1991 - val_acc: 0.9064
Epoch 28/200
4142/4142 [==============================] - 1s 280us/step - loss: 0.2538 - acc: 0.9049 - val_loss: 0.1656 - val_acc: 0.9361
Epoch 29/200
4142/4142 [==============================] - 1s 279us/step - loss: 0.2351 - acc: 0.9096 - val_loss: 0.1697 - val_acc: 0.9201
Epoch 30/200
4142/4142 [==============================] - 1s 282us/step - loss: 0.2395 - acc: 0.9055 - val_loss: 0.3191 - val_acc: 0.8584
Epoch 31/200
4142/4142 [==============================] - 1s 292us/step - loss: 0.2356 - acc: 0.9077 - val_loss: 0.1386 - val_acc: 0.9658
Epoch 32/200
4142/4142 [==============================] - 1s 280us/step - loss: 0.2233 - acc: 0.9147 - val_loss: 0.1441 - val_acc: 0.9680
Epoch 33/200
4142/4142 [==============================] - 1s 287us/step - loss: 0.2165 - acc: 0.9110 - val_loss: 0.1320 - val_acc: 0.9589
Epoch 34/200
4142/4142 [=======================

4142/4142 [==============================] - 1s 284us/step - loss: 0.2005 - acc: 0.9226 - val_loss: 0.1229 - val_acc: 0.9658
Epoch 37/200
4142/4142 [==============================] - 1s 282us/step - loss: 0.2016 - acc: 0.9243 - val_loss: 0.1263 - val_acc: 0.9566
Epoch 38/200
4142/4142 [==============================] - 1s 282us/step - loss: 0.1979 - acc: 0.9264 - val_loss: 0.1144 - val_acc: 0.9680
Epoch 39/200
4142/4142 [==============================] - 1s 282us/step - loss: 0.1905 - acc: 0.9282 - val_loss: 0.1463 - val_acc: 0.9452
Epoch 40/200
4142/4142 [==============================] - 1s 283us/step - loss: 0.1838 - acc: 0.9323 - val_loss: 0.1201 - val_acc: 0.9543
Epoch 41/200
4142/4142 [==============================] - 1s 281us/step - loss: 0.1880 - acc: 0.9272 - val_loss: 0.1364 - val_acc: 0.9543
Epoch 42/200
4142/4142 [==============================] - 1s 280us/step - loss: 0.2009 - acc: 0.9270 - val_loss: 0.1381 - val_acc: 0.9452
Epoch 43/200
4142/4142 [=======================

...

4142/4142 [==============================] - 1s 281us/step - loss: 0.0850 - acc: 0.9684 - val_loss: 0.0604 - val_acc: 0.9749
Epoch 125/200
4142/4142 [==============================] - 1s 274us/step - loss: 0.0761 - acc: 0.9736 - val_loss: 0.0605 - val_acc: 0.9795
Epoch 126/200
4142/4142 [==============================] - 1s 274us/step - loss: 0.0836 - acc: 0.9692 - val_loss: 0.0460 - val_acc: 0.9863
Epoch 127/200
4142/4142 [==============================] - 1s 284us/step - loss: 0.0749 - acc: 0.9716 - val_loss: 0.0952 - val_acc: 0.9566
Epoch 128/200
4142/4142 [==============================] - 1s 274us/step - loss: 0.0796 - acc: 0.9704 - val_loss: 0.0673 - val_acc: 0.9749
Epoch 129/200
4142/4142 [==============================] - 1s 276us/step - loss: 0.0797 - acc: 0.9743 - val_loss: 0.0728 - val_acc: 0.9726
Epoch 130/200
4142/4142 [==============================] - 1s 276us/step - loss: 0.0844 - acc: 0.9685 - val_loss: 0.1220 - val_acc: 0.9589
Epoch 131/200
4142/4142 [================

4142/4142 [==============================] - 1s 276us/step - loss: 0.0735 - acc: 0.9736 - val_loss: 0.0657 - val_acc: 0.9703
Epoch 134/200
4142/4142 [==============================] - 1s 280us/step - loss: 0.0736 - acc: 0.9721 - val_loss: 0.0593 - val_acc: 0.9726
Epoch 135/200
4142/4142 [==============================] - 1s 274us/step - loss: 0.0806 - acc: 0.9720 - val_loss: 0.0639 - val_acc: 0.9772
Epoch 136/200
4142/4142 [==============================] - 1s 280us/step - loss: 0.0714 - acc: 0.9739 - val_loss: 0.1143 - val_acc: 0.9680
Epoch 137/200
4142/4142 [==============================] - 1s 277us/step - loss: 0.0827 - acc: 0.9672 - val_loss: 0.1090 - val_acc: 0.9635
Epoch 138/200
4142/4142 [==============================] - 1s 282us/step - loss: 0.0756 - acc: 0.9736 - val_loss: 0.0692 - val_acc: 0.9726
Epoch 139/200
4142/4142 [==============================] - 1s 276us/step - loss: 0.0697 - acc: 0.9746 - val_loss: 0.0572 - val_acc: 0.9749
Epoch 140/200
4142/4142 [================

Epoch 142/200
4142/4142 [==============================] - 1s 275us/step - loss: 0.0773 - acc: 0.9716 - val_loss: 0.0626 - val_acc: 0.9840
Epoch 143/200
4142/4142 [==============================] - 1s 283us/step - loss: 0.0760 - acc: 0.9701 - val_loss: 0.0442 - val_acc: 0.9954
Epoch 144/200
4142/4142 [==============================] - 1s 271us/step - loss: 0.0697 - acc: 0.9725 - val_loss: 0.0434 - val_acc: 0.9909
Epoch 145/200
4142/4142 [==============================] - 1s 276us/step - loss: 0.0718 - acc: 0.9730 - val_loss: 0.0675 - val_acc: 0.9726
Epoch 146/200
4142/4142 [==============================] - 1s 273us/step - loss: 0.0748 - acc: 0.9748 - val_loss: 0.0438 - val_acc: 0.9954
Epoch 147/200
4142/4142 [==============================] - 1s 274us/step - loss: 0.0722 - acc: 0.9757 - val_loss: 0.0679 - val_acc: 0.9817
Epoch 148/200
4142/4142 [==============================] - 1s 271us/step - loss: 0.0728 - acc: 0.9711 - val_loss: 0.1019 - val_acc: 0.9680
Epoch 149/200
4142/4142 [==

4142/4142 [==============================] - 1s 275us/step - loss: 0.0642 - acc: 0.9759 - val_loss: 0.0856 - val_acc: 0.9726
Epoch 151/200
4142/4142 [==============================] - 1s 274us/step - loss: 0.0692 - acc: 0.9740 - val_loss: 0.2082 - val_acc: 0.9315
Epoch 152/200
4142/4142 [==============================] - 1s 276us/step - loss: 0.0730 - acc: 0.9722 - val_loss: 0.0743 - val_acc: 0.9726
Epoch 153/200
4142/4142 [==============================] - 1s 275us/step - loss: 0.0659 - acc: 0.9749 - val_loss: 0.0586 - val_acc: 0.9749
Epoch 154/200
4142/4142 [==============================] - 1s 277us/step - loss: 0.0678 - acc: 0.9732 - val_loss: 0.0553 - val_acc: 0.9726
Epoch 155/200
4142/4142 [==============================] - 1s 271us/step - loss: 0.0667 - acc: 0.9754 - val_loss: 0.0498 - val_acc: 0.9840
Epoch 156/200
4142/4142 [==============================] - 1s 269us/step - loss: 0.0665 - acc: 0.9740 - val_loss: 0.0974 - val_acc: 0.9726
Epoch 157/200
4142/4142 [================

4142/4142 [==============================] - 1s 279us/step - loss: 0.0683 - acc: 0.9756 - val_loss: 0.0951 - val_acc: 0.9658
Epoch 160/200
4142/4142 [==============================] - 1s 281us/step - loss: 0.0660 - acc: 0.9742 - val_loss: 0.0687 - val_acc: 0.9680
Epoch 161/200
4142/4142 [==============================] - 1s 272us/step - loss: 0.0659 - acc: 0.9767 - val_loss: 0.0524 - val_acc: 0.9817
Epoch 162/200
4142/4142 [==============================] - 1s 277us/step - loss: 0.0593 - acc: 0.9763 - val_loss: 0.0539 - val_acc: 0.9840
Epoch 163/200
4142/4142 [==============================] - 1s 274us/step - loss: 0.0640 - acc: 0.9778 - val_loss: 0.0492 - val_acc: 0.9886
Epoch 164/200
485/485 [==============================] - 0s 200us/step
Test score:  [0.06596696213195004, 0.977319587628866]


In [0]:
#Save model
now = time.time()
model_json = model.to_json()
with open('./drive/ColabNotebooks/Eye/trained_model/model_json_' + str(now) + '.json', 'w') as f:
  f.write(model_json)
f.close()
model.save_weights('./drive/ColabNotebooks/Eye/trained_model/weight_' + str(now) + '.h5', overwrite=True)